# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [58]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [59]:
weather_data = 'city_data.csv'
weather_df = pd.read_csv(weather_data)
weather_df.head()

,City,Cloudiness,Country,Date,Humidity,Lat,Lon,Max Temp,Wind Speed
0,hilo,1,US,1580178285,54,19.73,-155.09,80.60,13.87
1,aykhal,0,RU,1580178285,69,66.00,111.50,-11.20,4.47
2,butaritari,95,KI,1580178285,78,3.07,172.79,84.47,23.11
3,cidreira,77,BR,1580178286,77,-30.18,-50.21,74.30,12.68
4,cockburn town,25,TC,1580178286,76,21.46,-71.14,76.39,10.16


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [61]:
# Configuring gmaps with google key

gmaps.configure(api_key=g_key)

base_map = gmaps.figure()

# Setting lat_lon as Lat(itude) and Lon(gitude), as well as setting Humidity as a separate variable
lat_lon = weather_df[['Lat', 'Lon']].astype(float)
humidity = weather_df['Humidity'].astype(float)

# Generating the heat layer
heat_layer = gmaps.heatmap_layer(lat_lon, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

# Smooshing everything together all nicely
base_map.add_layer(heat_layer)
base_map

# I'm not entirely sure why it's showing the maps the way it does, however, and my Google-fu is weak

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [62]:
# Dropping null values
weather_df = weather_df.dropna()

# Refining weather data to: Max Temp > 70 (F), Max Temp < 80 (F), Wind Speed < 10 mph, and Cloudiness < 20%
hotel_df = weather_df[(weather_df['Max Temp'] > 70) & (weather_df['Max Temp'] < 80) & (weather_df['Wind Speed'] < 10) & (weather_df['Cloudiness'] < 20)] 
hotel_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lon,Max Temp,Wind Speed
30,san patricio,5,MX,1580178289,83,19.22,-104.70,75.20,5.82
50,pontes e lacerda,4,BR,1580178291,86,-15.23,-59.34,75.76,1.57
315,santa isabel do rio negro,15,BR,1580178320,90,-0.41,-65.02,75.65,1.95
325,maues,3,BR,1580178321,87,-3.38,-57.72,76.14,2.08
330,inongo,11,CD,1580178322,52,-1.95,18.27,70.27,3.15
379,tabou,16,CI,1580178328,90,4.42,-7.35,76.64,1.52
408,fougamou,0,GA,1580178331,95,-1.22,10.58,73.89,1.19
412,mandasa,4,IN,1580178332,73,18.87,84.47,73.40,4.83
547,barahona,1,DO,1580178347,74,18.21,-71.10,77.59,6.33
555,edd,0,ER,1580178348,77,13.93,41.69,72.16,4.29


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [78]:
# Adding the Hotel Name column
hotel_df.loc[:,('Hotel Name')] = ''
hotel_lat_lon = hotel_df[['Lat', 'Lon']].astype(str)
query = 'hotel'
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
    'location': hotel_lat_lon,
    'radius': 5000,
    'query': query,
    'key': g_key,
}
# use iterrows to iterate through pandas dataframe
for index, row in hotel_df.iterrows():
    # assemble url and make API request
    hotel_list = requests.get(base_url, params = params).json()
    
    # extract results
    results = hotel_list['results']#[0]['name']
    print(results)
'''
    try:
        hotel_df.loc[index, 'Hotel Name'] = results
       
    except (KeyError, IndexError):
        print("Missing Data: Skipping " + city)
        pass
'''    
hotel_df

[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]


,City,Cloudiness,Country,Date,Humidity,Lat,Lon,Max Temp,Wind Speed,Hotel Name
30,san patricio,5,MX,1580178289,83,19.22,-104.70,75.20,5.82,
50,pontes e lacerda,4,BR,1580178291,86,-15.23,-59.34,75.76,1.57,
315,santa isabel do rio negro,15,BR,1580178320,90,-0.41,-65.02,75.65,1.95,
325,maues,3,BR,1580178321,87,-3.38,-57.72,76.14,2.08,
330,inongo,11,CD,1580178322,52,-1.95,18.27,70.27,3.15,
379,tabou,16,CI,1580178328,90,4.42,-7.35,76.64,1.52,
408,fougamou,0,GA,1580178331,95,-1.22,10.58,73.89,1.19,
412,mandasa,4,IN,1580178332,73,18.87,84.47,73.40,4.83,
547,barahona,1,DO,1580178347,74,18.21,-71.10,77.59,6.33,
555,edd,0,ER,1580178348,77,13.93,41.69,72.16,4.29,


In [76]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[['Lat', 'Lon']]

In [77]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
base_map.add_layer(marker_layer)
# Display Map

base_map

Figure(layout=FigureLayout(height='420px'))